# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 10 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 10 stocks.

## Library Imports


In [2]:
import pandas as pd
import numpy as np
import requests
import math
from vnstock3 import *
import xlsxwriter
from scipy import stats

In [31]:
stock = Vnstock().stock(symbol= "ACB", source='VCI')

2024-09-26 23:46:56,637 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [4]:
stocks = stock.listing.symbols_by_group('VN30')
stock_df = pd.DataFrame(stocks)
stock_df.rename(columns = {'symbol': 'Ticker'}, inplace = True)
print(stock_df)

   Ticker
0     ACB
1     BCM
2     BID
3     BVH
4     CTG
5     FPT
6     GAS
7     GVR
8     HDB
9     HPG
10    MBB
11    MSN
12    MWG
13    PLX
14    POW
15    SAB
16    SHB
17    SSB
18    SSI
19    STB
20    TCB
21    TPB
22    VCB
23    VHM
24    VIB
25    VIC
26    VJC
27    VNM
28    VPB
29    VRE


In [47]:
stock = Vnstock().stock(symbol= "CTG", source='VCI')
# P/B Ratio
pb_ratio = stock.finance.ratio(period='quarter', lang='en').iloc[0,3]
# P/E Ratio
pe_ratio = stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,6]
# P/S Ratio
ps_ratio = stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,7]
# Enterprise Value
ev = (stock.finance.balance_sheet(period='year', lang='vi').iloc[0,-6] + stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,4] 
      - stock.finance.balance_sheet(period = 'quarter', lang = 'en').iloc[0,18])
# EV/ EBITDA
ev_ebitda = stock.finance.ratio(period='year', lang='vi').iloc[0,11]
# Gross Profit 
gp = stock.finance.income_statement(period='quarter', lang='en').iloc[0,20]
pb_ratio
pe_ratio
ps_ratio
ev
ev_ebitda


2024-09-27 00:32:56,987 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

0.3758402165601617

## Building a Value Strategy

- Các nhà đầu tư thường sử dụng một rổ số liệu định giá tổng hợp để xây dựng các chiến lược giá trị định lượng mạnh mẽ. Trong phần này, ta sẽ lọc các cổ phiếu có phân vị thấp nhất theo các số liệu sau:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)


In [59]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
final_df = pd.DataFrame(columns = rv_columns)
for ticker in stock_df['Ticker']:
    try:
        price = float(stock.quote.intraday(symbol=f'{ticker}', show_log=False).iloc[0, 1])

        stock = Vnstock().stock(symbol= ticker, source='VCI')
        # P/B Ratio
        pb_ratio = stock.finance.ratio(period='quarter', lang='en').iloc[0,3]
        # P/E Ratio
        pe_ratio = stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,6]
        # P/S Ratio
        ps_ratio = stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,7]
        # Enterprise Value
        ev = (stock.finance.balance_sheet(period='year', lang='vi').iloc[0,-6] + stock.finance.ratio(period = 'quarter', lang = 'en').iloc[0,4] 
            - stock.finance.balance_sheet(period = 'quarter', lang = 'en').iloc[0,18])
        # EV/ EBITDA
        ev_ebitda = stock.finance.ratio(period='year', lang='vi').iloc[0,11]
        # Gross Profit 
        gp = stock.finance.income_statement(period='quarter', lang='en').iloc[0,20]

        ev_gp = ev/gp

        new_row = pd.Series([ticker,price, 'N/A',pe_ratio,'N/A', pb_ratio,'N/A' ,ps_ratio, 'N/A', ev_ebitda,'N/A', ev_gp,'N/A', 'N/A'], index=rv_columns)

        final_df = pd.concat([final_df, pd.DataFrame([new_row])], ignore_index=True)

    except Exception as e:
        print(f"Error with ticker {ticker}: {e}")

final_df

2024-09-27 00:43:54,043 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.i

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ACB,26150.0,N/A,7.127993,N/A,1.564678,N/A,3.480515,N/A,0.642829,N/A,-3063.679714,N/A,N/A
1,BCM,72000.0,N/A,28.211642,N/A,3.986727,N/A,9.201868,N/A,30.629728,N/A,-167.440619,N/A,N/A
2,BID,50100.0,N/A,12.481376,N/A,2.198071,N/A,3.755991,N/A,0.383089,N/A,42.391573,N/A,N/A
3,BVH,43400.0,N/A,17.819479,N/A,1.444996,N/A,0.806387,N/A,0.045366,N/A,110.847280,N/A,N/A
4,CTG,36350.0,N/A,9.520017,N/A,1.431481,N/A,2.592254,N/A,0.375840,N/A,50.131742,N/A,N/A
5,FPT,134200.0,N/A,27.569014,N/A,7.163126,N/A,3.406599,N/A,16.479445,N/A,35.663103,N/A,N/A
6,GAS,74400.0,N/A,15.814019,N/A,2.485426,N/A,1.763029,N/A,10.651138,N/A,30.678203,N/A,N/A
7,GVR,36100.0,N/A,51.917437,N/A,2.803232,N/A,6.254215,N/A,29.029737,N/A,-1070.386767,N/A,N/A
8,HDB,28100.0,N/A,6.553097,N/A,1.566849,N/A,2.611790,N/A,0.453972,N/A,99.831158,N/A,N/A
9,HPG,26150.0,N/A,15.006368,N/A,1.543480,N/A,1.257406,N/A,12.496115,N/A,18.629242,N/A,N/A


## Calculating Value Percentiles


* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP


In [60]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in final_df.index:
    for metric in metrics.keys():
        final_df.loc[row, metrics[metric]] = stats.percentileofscore(final_df[metric], final_df.loc[row, metric])/100


for metric in metrics.values():
    print(final_df[metric])
   
final_df

0     0.133333
1     0.833333
2     0.433333
3     0.733333
4     0.333333
5     0.766667
6     0.633333
7     0.966667
8          0.1
9          0.5
10    0.066667
11         1.0
12    0.866667
13         0.6
14         0.8
15    0.666667
16    0.033333
17         0.4
18         0.7
19         0.2
20    0.233333
21         0.3
22    0.566667
23    0.266667
24    0.166667
25         0.9
26    0.933333
27    0.533333
28    0.466667
29    0.366667
Name: PE Percentile, dtype: object
0     0.533333
1     0.933333
2     0.633333
3     0.433333
4     0.366667
5          1.0
6          0.7
7     0.733333
8     0.566667
9          0.5
10    0.333333
11    0.833333
12         0.9
13    0.666667
14    0.066667
15         0.8
16    0.033333
17         0.4
18         0.6
19    0.266667
20    0.233333
21    0.166667
22    0.766667
23         0.1
24    0.466667
25         0.3
26    0.866667
27    0.966667
28         0.2
29    0.133333
Name: PB Percentile, dtype: object
0     0.733333
1          1.0


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ACB,26150.0,N/A,7.127993,0.133333,1.564678,0.533333,3.480515,0.733333,0.642829,0.466667,-3063.679714,0.033333,N/A
1,BCM,72000.0,N/A,28.211642,0.833333,3.986727,0.933333,9.201868,1.0,30.629728,0.966667,-167.440619,0.166667,N/A
2,BID,50100.0,N/A,12.481376,0.433333,2.198071,0.633333,3.755991,0.8,0.383089,0.266667,42.391573,0.633333,N/A
3,BVH,43400.0,N/A,17.819479,0.733333,1.444996,0.433333,0.806387,0.1,0.045366,0.066667,110.847280,0.9,N/A
4,CTG,36350.0,N/A,9.520017,0.333333,1.431481,0.366667,2.592254,0.533333,0.375840,0.2,50.131742,0.666667,N/A
5,FPT,134200.0,N/A,27.569014,0.766667,7.163126,1.0,3.406599,0.7,16.479445,0.8,35.663103,0.6,N/A
6,GAS,74400.0,N/A,15.814019,0.633333,2.485426,0.7,1.763029,0.3,10.651138,0.633333,30.678203,0.566667,N/A
7,GVR,36100.0,N/A,51.917437,0.966667,2.803232,0.733333,6.254215,0.9,29.029737,0.933333,-1070.386767,0.1,N/A
8,HDB,28100.0,N/A,6.553097,0.1,1.566849,0.566667,2.611790,0.6,0.453972,0.3,99.831158,0.866667,N/A
9,HPG,26150.0,N/A,15.006368,0.5,1.543480,0.5,1.257406,0.233333,12.496115,0.666667,18.629242,0.466667,N/A


## Calculating the RV Score
Bây giờ, ta sẽ tính RV Score (Robust Value), là điểm giá trị mà ta sẽ sử dụng để lọc cổ phiếu trong chiến lược này.

RV Score sẽ là mean của 4 percentile scores mà ta đã tính ở phần trước.

Để tính giá trị mean, chúng ta sẽ sử dụng Python's builtin statistics module.

In [61]:
from statistics import mean

for row in final_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(final_df.loc[row, metrics[metric]])
    final_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
final_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ACB,26150.0,N/A,7.127993,0.133333,1.564678,0.533333,3.480515,0.733333,0.642829,0.466667,-3063.679714,0.033333,0.38
1,BCM,72000.0,N/A,28.211642,0.833333,3.986727,0.933333,9.201868,1.0,30.629728,0.966667,-167.440619,0.166667,0.78
2,BID,50100.0,N/A,12.481376,0.433333,2.198071,0.633333,3.755991,0.8,0.383089,0.266667,42.391573,0.633333,0.553333
3,BVH,43400.0,N/A,17.819479,0.733333,1.444996,0.433333,0.806387,0.1,0.045366,0.066667,110.847280,0.9,0.446667
4,CTG,36350.0,N/A,9.520017,0.333333,1.431481,0.366667,2.592254,0.533333,0.375840,0.2,50.131742,0.666667,0.42
5,FPT,134200.0,N/A,27.569014,0.766667,7.163126,1.0,3.406599,0.7,16.479445,0.8,35.663103,0.6,0.773333
6,GAS,74400.0,N/A,15.814019,0.633333,2.485426,0.7,1.763029,0.3,10.651138,0.633333,30.678203,0.566667,0.566667
7,GVR,36100.0,N/A,51.917437,0.966667,2.803232,0.733333,6.254215,0.9,29.029737,0.933333,-1070.386767,0.1,0.726667
8,HDB,28100.0,N/A,6.553097,0.1,1.566849,0.566667,2.611790,0.6,0.453972,0.3,99.831158,0.866667,0.486667
9,HPG,26150.0,N/A,15.006368,0.5,1.543480,0.5,1.257406,0.233333,12.496115,0.666667,18.629242,0.466667,0.473333


## Selecting the 10 Best Value Stocks¶


In [62]:
final_df.sort_values(by = 'RV Score', inplace = True)
final_df = final_df[:10]
final_df.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy


In [63]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Nhập giá trị danh mục:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("Invalid \n Try again:")
        portfolio_size = input("Nhập giá trị danh mục:")

In [64]:
portfolio_input()

In [75]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(0, len(final_df['Ticker'])):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df['Price'][i])
final_df


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,STB,32900.0,30,7.534616,0.2,1.256095,0.266667,2.311940,0.366667,0.349696,0.133333,14.138275,0.366667,0.266667
1,SHB,10750.0,93,4.709521,0.033333,0.688778,0.033333,1.873637,0.333333,0.379809,0.233333,314.889434,1.0,0.326667
2,VHM,44300.0,22,8.252229,0.266667,0.999890,0.1,2.464423,0.5,7.990757,0.566667,-71.549632,0.233333,0.333333
3,TPB,16650.0,60,8.720823,0.3,1.157424,0.166667,2.344571,0.4,0.359140,0.166667,78.199734,0.766667,0.36
4,POW,13000.0,76,28.179979,0.8,0.967740,0.066667,1.102651,0.166667,8.198247,0.6,-102.261835,0.2,0.366667
5,ACB,26150.0,38,7.127993,0.133333,1.564678,0.533333,3.480515,0.733333,0.642829,0.466667,-3063.679714,0.033333,0.38
6,VRE,19150.0,52,9.670058,0.366667,1.086919,0.133333,4.169803,0.833333,7.361331,0.533333,-219.370746,0.133333,0.4
7,CTG,36350.0,27,9.520017,0.333333,1.431481,0.366667,2.592254,0.533333,0.375840,0.2,50.131742,0.666667,0.42
8,SSB,16950.0,58,10.076563,0.4,1.440484,0.4,4.207474,0.866667,0.520026,0.366667,-1562.001239,0.066667,0.42
9,MBB,25750.0,38,6.325365,0.066667,1.361737,0.333333,2.734865,0.633333,0.534507,0.4,65.221103,0.7,0.426667
